In [1]:
! pip install snowflake

In [2]:
import snowflake.connector
import config

def get_data(query, database=None, schema=None):
    # Establish the connection
    conn = snowflake.connector.connect(
        user=config.user,
        password= config.password,
        account= config.account,
        warehouse= config.warehouse,
        database= database if database else config.database,
        schema= schema if schema else config.schema
    )

    # Create a cursor object
    cursor = conn.cursor()

    cursor.execute(query)

    # Fetch and print the result
    result = cursor.fetchall()

    # Close the cursor and connection
    cursor.close()
    conn.close()
    return result

In [23]:
import pandas as pd

unique_cc = pd.read_excel('../VESSEL_PARTICULARS.xlsx', engine='openpyxl')['VESSEL CODE'].unique().tolist()
unique_cc_str = ', '.join([f"'{cc}'" for cc in unique_cc])

expense_query = f'''SELECT
    TO_VARIANT(DATE_TRUNC('MONTH', MS.POSTING_DATE))::VARCHAR AS "Year-Month",
    MS.COST_CENTER,
    MS.NODECODE AS "Categories",
    MS.ACCOUNT_CODE,
    MS.ACCOUNT_CODE_DESC AS "SUB_CATEGORIES",
    SUM(MS.AMOUNT_USD) AS "Expense"
    FROM
    CURATED_DB.VESSEL_ACCOUNTS.MANAGER_STATEMENT MS
    WHERE
    MS.BRANCHCODE = 'Operating Expenses' AND MS.COST_CENTER IN ({unique_cc_str}) and MS.PERIOD >= 202101 and MS.PERIOD <= 202312
    GROUP BY "Year-Month", MS.COST_CENTER, MS.NODECODE, MS.ACCOUNT_CODE, MS.ACCOUNT_CODE_DESC
    ORDER BY "Year-Month", MS.COST_CENTER, MS.NODECODE ASC'''

expense_data = pd.DataFrame(get_data(expense_query), columns=['DATE', 'COST_CENTER', 'CATEGORIES', "ACCOUNT_CODE", 'SUB_CATEGORIES', 'EXPENSE'])

In [24]:
expense_data

,DATE,COST_CENTER,CATEGORIES,ACCOUNT_CODE,SUB_CATEGORIES,EXPENSE
0,2021-01-01,2788,Administrative Expenses,5320-120,BANK CHARGES,20.34
1,2021-01-01,2788,Pre-Delivery Expenses,5164-135,PRE-OPS: STORES,2080.00
2,2021-01-01,2788,Pre-Delivery Expenses,5164-175,PRE-OPS: OTHERS,2041.22
3,2021-01-01,2788,Ship Software,5157-100,SHIP SOFTWARE,7000.00
4,2021-01-01,2817,Administrative Expenses,5320-120,BANK CHARGES,3.82
...,...,...,...,...,...,...
738632,2024-03-01,LIVI,Stores,5142-125,TRTMNT & MAINT CHEMICALS,308392.43
738633,2024-03-01,LIVI,Stores,5144-999,STORES - COMMITTED COST,-3953274.02
738634,2024-03-01,LIVI,Stores,5140-115,"PAINTS-SEASTOCK-HULL,DECK,ENG",193803.90
738635,2024-03-01,LIVI,Stores,5140-145,SIGNALS & SAFETY EQUIPMENTS,368719.25
